In [2]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

filename = "math sample for API testing.csv"

fields ={'Internal_ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [88]:
def process_file(filename, fields, tagging_df):
    process_fields = fields.keys()
    data = []
    bad_data = []
#    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(1):
            l = reader.next()

        for line in reader:
            error_test = False
            row = {}
            strand_dict = {}
            strand_list = []
            InternalID = int(line['Internal ID'])
            Strand = line['Strand'].split("; ")
            for strand in Strand:
                strand_sub_dict = {}
                content_focus_df = tagging_df['Content_Focus']
                StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
                strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID, 'Substrand': []}
                strand_list.append(strand_sub_dict)
                         
            substrand_list = line['Sub-Strand'].split("; ")
            strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)
                                 
            Differentiator_Keys = ['Shape', 'Units', 'Steps']            
            Differentiator_List = []
            for Differentiator in Differentiator_Keys:
                Differentiator_values = line[Differentiator].split("; ")
                Differentiator_sub_list = []
                Differentiator_df = tagging_df[Differentiator]
                DifferentiatorGUID_Name = Differentiator + "GUID"
                for value in Differentiator_values:
                    Differentiator_sub_dict = {}
                    DifferentiatorGUID, error_test = get_GUID_lite(value, Differentiator_df, error_test)
                    Differentiator_sub_dict = {Differentiator: value, DifferentiatorGUID_Name : DifferentiatorGUID}
                    Differentiator_sub_list.append(Differentiator_sub_dict)
                Differentiator_List.append(Differentiator_sub_list)
            #Process Differentiators
            
            
            row = {"Internal_ID": InternalID, "Organizers" : strand_list, "Differentiators": Differentiator_List}
            if error_test == True:
                bad_data.append(row)
            else: data.append(row)
            
            
    return data, bad_data

In [148]:
data, bad_data = process_file(filename, fields, tagging_dict)
pprint.pprint(bad_data)

[]


In [155]:
len(bad_data)
data= []

In [156]:
pprint.pprint(data)

[]


In [153]:
def processSubstrands(strand_list, substrand_list, df, error_test):
    GUID = None
    substrand_sublist = []
    if len(df[(df['Title']==lowercase(substrand_list[0]))]) == 0:  ####BUG: THIS JUST TESTS THE FIRST SUBSTRAND FOR EXISTENCE
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Substrand not found in master list'})
    elif len(substrand_list) < len(strand_list):
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: more strands than substrands'})
    elif len(strand_list) == 1: 
        for substrand in substrand_list:
            strand_list, error_test = getSubstrandGUID(strand_list[0]['Strand'], strand_list, substrand, df, error_test)    
    elif len(substrand_list) == 2 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 2)
    elif len(substrand_list) == 3 and len(strand_list) == 2:
        strand_list, error_test = matchStrandsSubstrands(strand_list, substrand_list, df, error_test, 3)
    else: 
        error_test = True
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Multiple substrands, cannot parse yet'})
    return strand_list, error_test

def getSubstrandGUID(strand, strand_list, substrand, df, error_test): 
    if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
        SubstrandGUID = df.ix[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand)), 'Unique ID'].item()
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'SubstrandGUID': SubstrandGUID, 'Content Focus': []})
    else: 
        strand_list[next(index for (index, d) in enumerate(strand_list) if d["Strand"] == strand)]['Substrand'].append({'Substrand': substrand, 'SubstrandGUID': 'Error: No Strand/Substrand match found in master list'})
        error_test = True
    return strand_list, error_test

    

In [154]:
strand_raw = "Geometry and Measurement; Algebra"
substrand_raw = "Shapes; Functions; Equations"
#print strand_raw
#print substrand_raw

error_test = False
strand_dict = {}
strand_list = []
temp_list = strand_raw.split("; ")
#print temp_list
for strand in temp_list:
    strand_sub_dict = {}
    #content_focus_df = tagging_df['Content_Focus']
    StrandGUID, error_test = get_GUID_lite(strand, content_focus_df, error_test)
    strand_sub_dict = {'Strand': strand, 'StrandGUID' : StrandGUID, 'Substrand': []}
    strand_list.append(strand_sub_dict)

#print strand_list
substrand_list = substrand_raw.split("; ")
#print substrand_list
strand_list, error_test = processSubstrands(strand_list, substrand_list, content_focus_df, error_test)

print strand_list


[{'Substrand': [{'SubstrandGUID': '15D1079A-8C17-11E6-81DC-467029C466BA', 'Substrand': 'Shapes', 'Content Focus': []}], 'StrandGUID': 'D66D958E-8BFB-11E6-921B-89CC29C466BA', 'Strand': 'Geometry and Measurement'}, {'Substrand': [{'SubstrandGUID': '1466B3D2-8C17-11E6-81DC-467029C466BA', 'Substrand': 'Functions', 'Content Focus': []}, {'SubstrandGUID': '14224210-8C17-11E6-81DC-467029C466BA', 'Substrand': 'Equations', 'Content Focus': []}], 'StrandGUID': 'D66A43AC-8BFB-11E6-921B-89CC29C466BA', 'Strand': 'Algebra'}]


In [102]:
#Case strand len = 1, substrand = 1



In [151]:
#Build a data frame to track match status to determine whether strand-substrand pairings can be deduced    
def matchStrandsSubstrands(strand_list, substrand_list, df, error_test, matchlimit):
    strand_names = []
    for strand in strand_list:
        strand_names.append(strand['Strand'])

    match_status = pd.DataFrame(False,index=strand_names,columns=substrand_list)

    for strand in strand_names:
        for substrand in substrand_list:
            if len(df[(df['Strand']==lowercase(strand)) & (df['Title']==lowercase(substrand))]) == 1:
                match_status.ix[strand,substrand] = True
    
    if int(match_status.sum().sum()) == matchlimit:
    #get the strand,substrand pairs where match_status is true  
        true_pairs = list(match_status[match_status == True].stack().index)
        for pair in true_pairs:
            strand_list, error_test = getSubstrandGUID(pair[0], strand_list, pair[1], df, error_test)
    else: 
        strand_list[0]['Substrand'].append({'Substrand': substrand_list[0], 'SubstrandGUID': 'Error: Ambiguous strand/substrand relationships'})
        error_test = True
    return strand_list, error_test
       


In [117]:
    #If there are 2 x 2 strand-substrand possibilities with 2 matches, then nothing need be deduced

    
#elif match_status.sum().sum() == 3:
    #figure out what matches
    
    #Output the substrand underneath the appropriate strand.

IndentationError: expected an indented block (<ipython-input-117-8d412881bef4>, line 4)

In [5]:
def lowercase(text):
    return unicode(text, "latin2").lower()

In [8]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    df_lowercase(data, 'Title')
    if len(data.columns) == 6:
        df_lowercase(data, 'Strand')
        df_lowercase(data, 'Sub-Strand')
    if len(data.columns) == 7:
        df_lowercase(data, 'Strand')
    return data

In [9]:
def df_lowercase(dataframe, column):
    lowercase = lambda x: unicode(x, "latin2").lower()
    dataframe[column] = dataframe[column].apply(lowercase)
    return dataframe

In [10]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
number_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")
content_focus_df = Load_GUIDs("Math_Content_Foc.csv")
component_df = Load_GUIDs("Math_Components.csv")
tagging_dict = {'Steps': steps_df, 'Shape': shapes_df, 'Number': number_df, 'Units': units_df, 'Content_Focus': content_focus_df, 'Component': component_df}

In [11]:

def get_GUID_lite(tag, df, error_test): 
    #get GUID for steps, number, units, shape
    tag = unicode(tag, "latin2").lower()
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        if tag == r'n/a':
            GUID = r'N/A'
        else: 
            GUID = "Error: no matches found"
            error_test = True
    else: 
        GUID = "Error: resolve multiple values"
        error_test = True
    return GUID, error_test

In [ ]:
number_test = "Square roots"
error_test = False
print get_GUID_lite(number_test, number_df, error_test)

In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument

tag = 'N/A'
if tag == r'N/A':
    print tag


In [ ]:
'''
Data structure

{Internal ID: value
    {Organizers: [
        {Strand: [ {Strand: Strand_value,
                   Strand_GUID: GUID,
                   Substrand: [{Substrand: Substrand_value,
                                Substrand_GUID: GUID,
                                Content_Focus: [{Content_Focus: Content_Focus_value,                       
                                                Content_Focus_GUID: GUID,
                                                Component: [{Component: Component_value,
                                                              Component_GUID: GUID}]}]}]}]
        {Descriptors: {Number: [N1, N2, ...]}
                        {Digits: [D1, D2, ...]}
            {Shape: [Shape1, ...]}
            {Units: [Units1, ...]}
            {Steps: Steps}
            {Context: Context}
            {SIG Stem: [Sig Stem1, ...]}
            {SIG Options: [SIG Options1, ...]}
            {Additional Modifiers: [AM1, ...]}
}

'''